In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from pyspark.ml.evaluation import BinaryClassificationEvaluator as BCE
from pyspark.ml.classification import MultilayerPerceptronClassifier

import week4 as w4

## 3. Modelo de Red Neuronal

Una de las tipologías de algoritmos más utilizadas para la clasificación de imágenes es la Red Neuronal. En esta sección vamos a utilizar una para intentar obtener una fiabilidad más alta.

In [2]:
#leemos los datos
df_raw = sqlContext.read.parquet('pca_features.parquet')
df = df_raw.withColumn('label', (df_raw['target'] - 1).astype('double'))['dr7objid', 'pca_features', 'label']

### Partición Test - Train

In [3]:
(df_train, df_test) = df.randomSplit([0.7, 0.3], seed=1234)

### Entrenamiento

Primero tenemos que definir la estructura de nuestra red neuronal con:

+ 64 nodos de entrada
+ 16 y 8 nodos en las capas ocultas
+ 2 nodos de salida

In [28]:
layers = [64, 16, 8, 2] 

Creamos la red neuronal y la entrenamos.

In [29]:
mlp = MultilayerPerceptronClassifier(labelCol="label",
                                     featuresCol="pca_features", 
                                     maxIter=100, 
                                     layers=layers, 
                                     seed=123)

mlp_model = mlp.fit(df_train)

### Validación

In [32]:
pred_df = mlp_model.transform(df_test)
cm = w4.confussion_matrix(pred_df, 'label', 'prediction').show()

Recall / Sensitivity / TPR =  0.9536903039073806
Specificity = 1 - FPR =  0.9340659340659341
Precision =  0.9648609077598829
Prevalence =  0.6549763033175355
Accuracy =  0.9469194312796209
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       0.0|  340|
|  0.0|       1.0|   24|
|  1.0|       0.0|   32|
|  1.0|       1.0|  659|
+-----+----------+-----+



Vemos que el porcentaje de aciertos se ha incrementado con respecto al algoritmo de regresión logística. Ahora tenemos cerca de un 95% de aciertos.

# Conclusiones

A pesar de que no podemos hacer una comparación directa de los parámetros que utilizamos para ambos algoritmos, el de Regresión Logística y Red Neuronal, la Red Neuronal parece ser el mejor candidato sobre el cual afinar nuestro modelo.


Para dicha afinación se pueden ir desde lo más basico como un filtrado más detallado de los datos hasta modificar el número de iteraciones o nodos en las capas del algoritmo.


Como trabajo a futuro, el contraste de las fotos puede ser incrementado para una mejor distinción entre formas.

